In [ ]:
#Energy need for all buildings - We are going to transform it into district heating need and electricity need

#libraries import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob # library for reading all files in the folder
from matplotlib.lines import Line2D
import seaborn as sns

print('Libraries imported')
##########################################################################################
##########################################################################################
##########################################################################################
Data = pd.read_csv('Data_Baseline.csv')
Coupling = Data[['Hour', 'Month', 'Electricity [MWh]','PV [MWh]', 'CO2', 'Price']].copy()
# Electricity balance 
Coupling['Coupling'] =  Coupling['PV [MWh]'] - Coupling['Electricity [MWh]']
Coupling['Excess'] = np.where(Coupling['Coupling']> 0, Coupling['Coupling'], 0)
Coupling['Defect'] = np.where(Coupling['Coupling'] <= 0, (-1) *Coupling['Coupling'], 0)
print('Data read')

##########################################################################################
##########################################################################################
##########################################################################################
# Variable generation
# ## Charge and discharge of batteries
# Battery Characteristics: 

Cmax = 1.4 # Maximum charge of 1.2 MWh
Pmax = 1 # Maximum charge per hour = 0.25 MWh
Co = 0 # initial charge state
eff = np.sqrt(0.95) # Charge efficiency

# I need to iterate over values, so I will extract the columns: 
Demand = Coupling['Electricity [MWh]'].to_numpy() # Demand
PV = Coupling['PV [MWh]'].to_numpy() # PV electricity 
Balance = Coupling['Coupling'].to_numpy() # PV - Electricity demand
Excess = Coupling['Excess'].to_numpy() # Excess PV electricity
Defect = Coupling['Defect'].to_numpy() # Lack of PV electricity
Charge = np.zeros(len(Coupling)) # battery charge 
Export = np.zeros(len(Coupling)) # Exported electricity 
To_battery =  np.zeros(len(Coupling)) # electricity going to battery
From_battery = np.zeros(len(Coupling)) # Electricity taken from battery
SOC = np.zeros(len(Coupling)) # state of charge


##########################################################################################
##########################################################################################
##########################################################################################
# Battery model
for i in range(len(Coupling)):
    
    # For when the battery is charging
    # I have to make sure that Charge[0] exists
    if ((Balance[i] > 0) and (np.where(i == 0, 0, Charge[i-1])<=Cmax)): 
        Cbq = Cmax - Charge[i-1] # Check the battery left to be charged
        PVb = min(Excess[i], Cbq, Pmax) # Electricity going to the battery
        To_battery[i] = PVb             # Keep track of that value
                     # charge at each time step
        Charge[i] = np.where(i == 0, Charge[i] + To_battery[i] * eff, Charge[i-1] + To_battery[i]*eff)
        Export[i] = Excess[i] - PVb 
        SOC[i] = Charge[i]/Cmax
        
        #For when there is excess PV but the battery is full
    elif ((Balance[i] > 0) and (Charge[i-1] == Cmax)):
        Export[i] = Excess[i]
        Charge[i] = Charge[i-1]
        
        
    #For when battery input is needed
    elif (Balance[i] <= 0):
        Bq = min(Defect[i], Pmax, np.where(i == 0, 0, Charge[i-1]*eff))
        Charge[i] =  np.where(i == 0, 0, Charge[i-1] - Bq/eff)
        From_battery[i] = Bq
        SOC[i] = Charge[i]/Cmax

##########################################################################################
##########################################################################################
##########################################################################################
# Data preparation for analysis

Grid_PEF = 2.56
Solar_PEF = 1.0
Battery_data = pd.DataFrame({
    'Hour' : Data['Hour'],
    'Month' : Data['Month'],
    'Electricity price' : Data['Price'],
    'Demand [MWh]' : Data['Electricity [MWh]'],
    'Grid Electricity [MWh]' : From_Grid,
    'PV Electricity [MWh]' : Data['PV [MWh]'],
    'PV Consumption [MWh]' : From_PV,
    'PV Export [MWh]' : Export,
    'Battery-supplied Electricity [MWh]' :From_battery,
    'CO2 Emissions [kg]' : From_Grid*Data['CO2'],
    'CO2 Avoided [kg]' : (From_PV + From_battery)*Data['CO2'],
    'CO2 Saved Exports [kg]' : Export * Data['CO2'],
    'Cost Electricity' : From_Grid * Data['Price'] * 1000,
    'Cost saved PV' : (From_PV + From_battery)*Data['Price'] * 1000,
    'Return Exports' : Export * Data['Price'] * 1000,
    'PE Grid' : From_Grid*Grid_PEF,
    'PE Solar' : (From_PV + From_battery) * Solar_PEF,
    'PE Exports' : Export * Solar_PEF,
    'SOC' : SOC
})
Battery_data['PE Final Balance'] = Battery_data['PE Grid'] + Battery_data['PE Solar'] - Battery_data['PE Exports']

months = Battery_data['Month'].unique()
Battery_monthly = Battery_data.groupby(['Month']).sum().reindex(months).drop('Hour', axis = 1)
Battery_monthly['Months'] = months

##########################################################################################
##########################################################################################
##########################################################################################
# Export

Battery_data.to_csv('Battery_hourly.csv')
Battery_monthly.to_csv('Battery_monthly.csv')